In [2]:
import sys
sys.path.append('../thinkbayes2/')

In [23]:
from thinkbayes2 import Pmf, Suite

In [7]:
pmf = Pmf()
for x in range(1, 7):
    pmf.Set(x, 1/6)

In [8]:
print(pmf.Prob(4))

0.16666666666666666


# the cookie problem

In [11]:
pmf = Pmf()
pmf.Set('Bowl 1', 0.5)
pmf.Set('Bowl 2', 0.5)

In [12]:
pmf.Mult('Bowl 1', 0.75)
pmf.Mult('Bowl 2', 0.5)
pmf.Normalize()

0.625

In [13]:
print(pmf.Prob('Bowl 1'))

0.6000000000000001


In [69]:
class Cookie(Pmf):
    
    def __init__(self, mixes):
        self.mixes = mixes
        Pmf.__init__(self)
        for h in mixes.keys():
            self.Set(h, 1)
        self.Normalize()
    
    def update(self, data):
        for hypo in self.Values():
            likelihood = self.likelihood(data, hypo)
            self.Mult(hypo, likelihood)
        self.Normalize()
    
    def likelihood(self, data, hypo):
        mix = self.mixes[hypo]
        return mix[data]
        

In [70]:
mixes = {'Bowl 1': dict(vanilla=0.75, chocolate=0.25), 
         'Bowl 2': dict(vanilla=0.5, chocolate=0.5)}
cookie = Cookie(mixes)
cookie.update('vanilla')
for hypo, prob in cookie.Items():
    print('%s: %f'%(hypo, prob))

Bowl 1: 0.600000
Bowl 2: 0.400000


# Monty Hall problem

In [53]:
class Monty(Suite):
    def Likelihood(self, data, hypo):
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

In [52]:
suite = Monty('ABC')
suite.Update('B')
suite.Print()

test
test
test
A 0.3333333333333333
B 0.0
C 0.6666666666666666


# M&M problem

In [63]:
class MandMs(Suite):
    def __init__(self, hypos):
        Suite.__init__(self, hypos)
        self.mix94 = dict(brown=30,
                     yellow=20,
                     red=20,
                     green=10,
                     orange=10,
                     tan=10)
        self.mix96 = dict(blue=24,
                     green=20,
                     orange=16,
                     yellow=14,
                     red=13,
                     brown=13)
        hypoA = dict(bag1=self.mix94, bag2=self.mix96)
        hypoB = dict(bag1=self.mix96, bag2=self.mix94)
        self.hypotheses = {hypos[0]: hypoA, hypos[1]: hypoB}

    def Likelihood(self, data, hypo):
        h = self.hypotheses[hypo]
        l = h[data[0]][data[1]]
        return l

In [64]:
mm = MandMs('AB')
mm.Update(['bag1', 'yellow'])
mm.Update(['bag2', 'green'])

mm.Print()

A 0.7407407407407407
B 0.2592592592592592


# Exercise 2.1

In [93]:
class Bowl:
    
    def __init__(self, cookies):
        self.cookies = cookies
    
    def draw(self, flavor):
        if flavor in self.cookies:
            drawprob = self.cookies[flavor]/self.nmix()
            self.cookies[flavor] -= 1
            if self.cookies[flavor] == 0:
                self.cookies.pop(flavor)
            return drawprob
        else:
            return 0

    def nmix(self):
        return sum(self.cookies.values())
    
    def __str__(self):
        return str(self.cookies)

    
class Cookie(Suite):
    
    def __init__(self, bowls):
        self.bowls = bowls
        Suite.__init__(self, bowls.keys())

    def Likelihood(self, data, hypo):
        l = self.bowls[hypo].draw(data)
        return l
        

In [95]:
bowl1 = Bowl(dict(vanilla=30, chocolate=10))
bowl2 = Bowl(dict(vanilla=20, chocolate=20))

cookie = Cookie(dict(bowl1=bowl1, bowl2=bowl2))
cookie.Update('vanilla')
cookie.Print()
cookie.Update('chocolate')
cookie.Print()
for i in range(9):
    cookie.Update('chocolate')
    cookie.Print()
    print(bowl1)

bowl1 0.6000000000000001
bowl2 0.4
bowl1 0.4285714285714286
bowl2 0.5714285714285714
bowl1 0.2621359223300971
bowl2 0.7378640776699028
{'vanilla': 29, 'chocolate': 8}
bowl1 0.13636363636363635
bowl2 0.8636363636363635
{'vanilla': 29, 'chocolate': 7}
bowl1 0.06104651162790699
bowl2 0.9389534883720931
{'vanilla': 29, 'chocolate': 6}
bowl1 0.023800528900642243
bowl2 0.9761994710993579
{'vanilla': 29, 'chocolate': 5}
bowl1 0.008061420345489446
bowl2 0.9919385796545106
{'vanilla': 29, 'chocolate': 4}
bowl1 0.002316602316602318
bowl2 0.9976833976833978
{'vanilla': 29, 'chocolate': 3}
bowl1 0.0005355548943766738
bowl2 0.9994644451056234
{'vanilla': 29, 'chocolate': 2}
bowl1 8.929900282780178e-05
bowl2 0.9999107009971722
{'vanilla': 29, 'chocolate': 1}
bowl1 8.118750253710948e-06
bowl2 0.9999918812497463
{'vanilla': 29}


- bowl1: 30 vanilla, 10 choco
- bowl2: 20 vanilla, 20 choco

P(b1 | vanilla) = (P(b1)*P(vanilla | b1))/P(vanilla)

first draw:
- = (40/80 * 30/40)/(50/80)
- = (1/2 * 3/4)/(5/8)
- = (3/8)/(5/8)
- = 3/5

state:
- bowl1: 29 vanilla, 10 choco
- bowl2: 19 vanilla, 20 choco

second draw:
- = (39/78 * 29/39)/(49/78)
- = ((39*29)/(78*39))/(49/78)
- = (29/78)/(49/78)
- = 29/49

In [87]:
(39/79 * 29/39)/(49/79)

0.5918367346938775